In [13]:
import math, rasterio as rio, geopandas as gpd
from rasterio.enums import Resampling
from rasterio.mask import mask

In [3]:
def calculate_Aexp(r_p, r_uav, h_p, theta):
    """
    Calcola l'area esposta all'impatto (Aexp) con la formula corretta.
    :param r_p: Raggio medio di una persona (metri).
    :param r_uav: Raggio del UAV (metri).
    :param h_p: Altezza media di una persona (metri).
    :param theta: Angolo di impatto al suolo (radianti).
    :return: Area esposta (metri quadrati).
    """
    return 2 * (r_p + r_uav) / (h_p * math.tan(theta)) + math.pi * (r_p + r_uav)**2

def calculate_Pimpact(rho_xy, Aexp):
    """
    Calcola la probabilità di impatto su una persona (Pimpact).
    :param rho_xy: Densità della popolazione nella posizione (x, y) (persone per metro quadrato).
    :param Aexp: Area esposta all'impatto (metri quadrati).
    :return: Probabilità di impatto su una persona.
    """
    return rho_xy * Aexp

In [18]:
%%time

with rio.open('Census_data/masked_pop_density_mask.tif') as dataset:
    # Leggi la prima banda (assumendo una sola banda per la densità abitativa)
    density_data = dataset.read(1, resampling=Resampling.bilinear)

Pimpact_raster = density_data.copy()

r_p = 0.3  # es. raggio medio di una persona in metri
r_uav = 0.5  # es. raggio del UAV in metri
h_p = 1.7  # es. altezza media di una persona in metri
theta = math.radians(45)  # es. angolo di impatto in gradi convertito in radianti

# Calcolo di Pimpact per ogni pixel

for i in range(density_data.shape[0]):
    for j in range(density_data.shape[1]):
        #if i % 10000 == 0 or j % 10000 == 0:
         #   print(f"Elaborati {i} su {density_data.shape[0]} i e elaborati {j} su {density_data.shape[1]} j")
        rho_xy = density_data[i, j]
        Aexp = calculate_Aexp(r_p, r_uav, h_p, theta)
        Pimpact_raster[i, j] = calculate_Pimpact(rho_xy, Aexp)

# Salvataggio del nuovo raster di Pimpact

with rio.open(
    'Census_data/Pimpact_raster.tif', 'w', 
    driver='GTiff',
    height=Pimpact_raster.shape[0],
    width=Pimpact_raster.shape[1],
    count=1,
    dtype=Pimpact_raster.dtype,
    crs=dataset.crs,
    transform=dataset.transform
) as new_dataset:
    new_dataset.write(Pimpact_raster, 1)

CPU times: total: 11min 54s
Wall time: 19min 43s


In [19]:
# add the administrative borders paths
province_borders_path = "Limiti01012023/ProvCM01012023/ProvCM01012023_WGS84.shp"
region_borders_path = "Limiti01012023/Reg01012023/Reg01012023_WGS84.shp"

# load the administrative borders
region_borders = gpd.read_file(region_borders_path)
province_borders = gpd.read_file(province_borders_path)

#filter vector data for the area of interest
emilia = region_borders[region_borders['DEN_REG'] == 'Emilia-Romagna']

In [20]:
def mask_raster_with_geodataframe(raster_path, gdf, output_path):
    # Carica il raster
    with rio.open(raster_path) as src:
        # Converti le geometrie del GeoDataFrame in formati accettati da rio
        geometries = [feature["geometry"] for _, feature in gdf.iterrows()]
        out_image, out_transform = mask(src, geometries, crop=True)

        # Salva il raster mascherato
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        with rio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)


# Esempio di utilizzo
mask_raster_with_geodataframe('Census_data/Pimpact_raster.tif', emilia,
                              'Census_data/Pimpact_raster_mask.tif')

In [ ]:
import numpy as np
import rasterio

# Funzione per calcolare il valore atteso di una variabile raster ponderata dal PDF
def expected_value(raster_data, PDF):
    """
    Calcola il valore atteso ponderato di una variabile raster.
    
    :param raster_data: array NumPy del raster della variabile di interesse.
    :param PDF: array NumPy del raster della funzione di densità di probabilità.
    :return: valore atteso ponderato della variabile.
    """
    return np.sum(raster_data * PDF)

# Funzione per caricare i raster e calcolare Pfatality
def calculate_Pfatality_from_rasters(pdf_path, e_imp_path, s_path, alpha, beta):
    """
    Calcola la probabilità di fatalità a partire dai raster della densità di probabilità (PDF),
    dell'energia di impatto (E_imp) e del fattore di protezione (S).
    
    :param pdf_path: percorso al file raster del PDF.
    :param e_imp_path: percorso al file raster dell'energia di impatto.
    :param s_path: percorso al file raster del fattore di protezione.
    :param alpha: energia di impatto per una probabilità di fatalità del 50% quando S=6.
    :param beta: energia di impatto per causare una fatalità con S vicino a zero.
    :return: array NumPy della probabilità di fatalità.
    """
    # Carica il raster PDF
    with rasterio.open(pdf_path) as pdf_dataset:
        PDF = pdf_dataset.read(1)
    
    # Carica il raster dell'energia di impatto
    with rasterio.open(e_imp_path) as e_imp_dataset:
        E_imp = e_imp_dataset.read(1)
    
    # Carica il raster del fattore di protezione
    with rasterio.open(s_path) as s_dataset:
        S = s_dataset.read(1)
    
    # Calcola i valori attesi per E_imp e S
    E_E_imp = expected_value(E_imp, PDF)
    E_S = expected_value(S, PDF)
    
    # Calcola k e Pfatality per ogni cella
    k = np.minimum(1, (beta / E_E_imp) ** (3 / E_S))
    Pfatality = (1 - k) / (1 - 2 * k + (np.sqrt(alpha / beta) * (beta / E_E_imp) ** (3 / E_S)))
    
    return Pfatality

# Percorsi dei file raster
pdf_path = 'path/to/pdf_raster.tif'  # Percorso al file raster della funzione di densità di probabilità
e_imp_path = 'path/to/e_imp_raster.tif'  # Percorso al file raster dell'energia di impatto
s_path = 'path/to/s_raster.tif'  # Percorso al file raster del fattore di protezione

# Valori noti per alpha e beta
alpha = 60  # Energia di impatto per una probabilità di fatalità del 50% quando S=6
beta = 34   # Energia di impatto per causare una fatalità con S vicino a zero

# Calcola Pfatality
Pfatality_raster = calculate_Pfatality_from_rasters(pdf_path, e_imp_path, s_path, alpha, beta)

# Salva il raster risultante di Pfatality
output_path = 'path/to/output_pfatality_raster.tif'  # Percorso al file raster di output per Pfatality
with rasterio.open(pdf_path) as src:
    profile = src.profile  # Copia il profilo del file sorgente per mantenere i metadati
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(Pfatality_raster.astype(rasterio.float32), 1)  # Scrive il raster di Pfatality nel file di output